In [2]:
import pandas as pd

# on charge le fichier avec le train
df = pd.read_csv("../Data/processed/train.csv")

# on enleve les lignes qui nous semble inutile
df = df[df['niveau_pollution'] != 'pas de données']
df = df[['Nom de la Station', 'Nom de la ligne']].drop_duplicates()

# on Groupe par ligne pour reconstituer les connections
lines = df.groupby('Nom de la ligne')['Nom de la Station'].apply(list)

# on genere  les connexions directes sur chaque ligne
connections = []
for stations in lines:
    for i in range(len(stations) - 1):
        s1 = stations[i]
        s2 = stations[i + 1]
        connections.append((s1, s2))
        connections.append((s2, s1))  #on detecte les liens B->A pour les supprimer car ce son des doublons

# On constuit la bdd
df_conn = pd.DataFrame(connections, columns=['station1', 'station2'])
df_conn['trajet_existe'] = True

# on utilise la ligne pour supprimer les doublons
df_conn_reduit = df_conn[df_conn['station1'] < df_conn['station2']]

#csv
df_conn_reduit.to_csv("graphe_stations_reduit.csv", index=False)

print("Fichier généré : graphe_stations_reduit.csv")
print("Nombre de connexions uniques :", len(df_conn_reduit))


Fichier généré : graphe_stations_reduit.csv
Nombre de connexions uniques : 241


In [ ]:
import networkx as nx

G = nx.from_pandas_edgelist(df_conn[df_conn['trajet_existe']], 'station1', 'station2')
nx.draw(G, with_labels=True, node_size=10, font_size=6)
